## Downloader

### usdt symbols

In [9]:
import nest_asyncio
nest_asyncio.apply()

In [10]:
from downloader import Downloader
downloader = Downloader()

In [13]:
await downloader.download_trading_symbols()

Saved trading_symbols_2024-09-15_by_launch.csv
Saved trading_symbols_2024-09-15_by_size.csv


In [ ]:
import yaml
with open('configs.yaml', 'r') as file:
    configs = yaml.safe_load(file)
    
last_launch_time = configs['last_launch_time']
exclude_symbols = configs['exclude_symbols']
symbols = downloader.load_trading_symbols(last_launch_time, exclude_symbols)
print(len(symbols))
symbols

### fetch recent klines

In [ ]:
from downloader import Downloader
downloader = Downloader()

In [ ]:
symbols = downloader.load_usdt_symbols()['Symbol'].tolist()
print(len(symbols))
# symbols = ["BTCUSDT", "ETHUSDT", "XRPUSDT", "ADAUSDT", "DOGEUSDT"]
intervals = ['1 month', '1 week', '1 day', '6 hours', '1 hour']

232


In [ ]:
processed_results, klines_complete, incomplete_klines = await downloader.fetch_recent_klines(symbols, intervals)
print(len(processed_results), klines_complete, len(incomplete_klines))

Fetching recent klines:   0%|          | 0/1160 [00:00<?, ?it/s]

Fetching recent klines: 100%|██████████| 1160/1160 [00:12<00:00, 93.56it/s]


1160 True 0


In [ ]:
# processed_results[('BTCUSDT', '1 hour')]

## Processor

### processing klines

In [1]:
from processor import Processor
processor = Processor()

In [2]:
import yaml
with open('configs.yaml', 'r') as file:
    configs = yaml.safe_load(file)
    
intervals = configs['intervals']
# intervals = ["15 minutes"]
processor.create_env_data(intervals=intervals, reprocess=True, recalculate_constants=False, batch_size=None)

Processing interval: 1 month


Loading data: 100%|██████████| 231/231 [00:02<00:00, 86.47it/s]


Processing summary: Full: 231, New data: 0, Up-to-date: 0


Saving: 100%|██████████| 231/231 [00:03<00:00, 72.26it/s]


Processing interval: 1 week


...

Saving: 100%|██████████| 231/231 [00:06<00:00, 35.13it/s]


Processing interval: 1 hour


Loading data: 100%|██████████| 231/231 [00:12<00:00, 18.55it/s]


Processing summary: Full: 231, New data: 0, Up-to-date: 0


Saving: 100%|██████████| 231/231 [00:16<00:00, 13.60it/s]


In [ ]:
processed_klines, processing_issues = processor.process_recent_klines(processed_results)

In [ ]:
processed_klines.keys()

## Plotter

### plotting raw klines

In [ ]:
from plotter import Plotter
plotter = Plotter()

Main klines data folder created: ../data/klines
Interval folders created successfully.


In [ ]:
symbol = "SHIB1000USDT"
interval = "1 day"
df = plotter.load_data(symbol, interval)

In [ ]:
plotter.plot_price_history_and_stats(df, symbol)

### plotting processed klines

In [2]:
from plotter import Plotter
plotter = Plotter()

In [ ]:
potential_stablecoins = plotter.identify_potential_stablecoins(threshold=0.05, interval="1 day")

In [3]:
# intervals = ["1 month", "1 week", "1 day", "6 hours", "1 hour", "15 minutes"]
intervals = ["15 minutes"]

for interval in intervals:
    plotter.plot_efficient_symbol_comparison(
        interval=interval,
        num_symbols=None,
        use_processed=True,
        use_all_symbols=True,
        exclude_symbols=['USDCUSDT'],
        show_all_plots=True,
    )

Plotting data: 100%|██████████| 231/231 [01:00<00:00,  3.81it/s]


Symbols comparison plot saved as '../data/plots/all_symbols_processed_15_minutes.png'


In [ ]:
plotter.plot_recent_data(processed_klines)

Recent data comparison plot for 1 week saved as '../data/plots/recent/recent_data_comparison_1_week.png'
Recent data comparison plot for 6 hours saved as '../data/plots/recent/recent_data_comparison_6_hours.png'
Recent data comparison plot for 1 month saved as '../data/plots/recent/recent_data_comparison_1_month.png'
Recent data comparison plot for 1 day saved as '../data/plots/recent/recent_data_comparison_1_day.png'
Recent data comparison plot for 1 hour saved as '../data/plots/recent/recent_data_comparison_1_hour.png'


## Actor

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from actor import Actor
actor = await Actor().initialize()

In [202]:
processed_klines, klines_complete, incomplete_klines, processing_issues, last_klines = await actor.fetch_and_process_klines(intervals_back=24*30 + 10)
combined_klines = actor.combine_recent_klines(processed_klines)
observations = actor.create_observation(combined_klines)

Fetching recent klines: 100%|██████████| 5/5 [00:00<00:00, 15.07it/s]


In [203]:
processed_klines  # 1726437600000

{('BTCUSDT',
  '1 month'): {'data': array([[1.1493431 , 1.1767172 , 0.99785787, 1.07619   , 1.6624483 ],
         [1.07619   , 1.0789348 , 0.8195351 , 0.8734317 , 1.6804976 ],
         [0.8734317 , 0.9666315 , 0.8476486 , 0.94120127, 1.6575767 ],
         [0.94120127, 0.97540575, 0.864302  , 0.8761476 , 1.6177814 ],
         [0.8761476 , 0.931843  , 0.83170104, 0.86203676, 1.6389761 ],
         [0.86203676, 0.8983049 , 0.8310046 , 0.8853524 , 1.5869495 ],
         [0.8853524 , 0.90585446, 0.76266974, 0.8081979 , 1.586553  ],
         [0.8081979 , 0.83834434, 0.78361785, 0.79280734, 1.4707441 ],
         [0.79280734, 0.953618  , 0.7917958 , 0.9381981 , 1.5851257 ],
         [0.9381981 , 0.977076  , 0.9029624 , 0.9381962 , 1.5989679 ],
         [0.9381962 , 1.0393636 , 0.86483586, 1.0281367 , 1.6885499 ],
         [1.0281367 , 1.0662477 , 1.0043072 , 1.0396581 , 1.6417238 ],
         [1.0396581 , 1.0487194 , 0.9850357 , 1.0085753 , 1.608207  ],
         [1.0085753 , 1.0715603 , 0.968115 

In [204]:
actions, action_probs, q_action, v = actor.get_action_values(observations)

In [205]:
df = actor.create_action_summary(actions, action_probs, q_action, v, last_klines, create_df=True)

In [206]:
len(df)

1

In [207]:
df.head(10)

,Symbol,action,action_prob,q_action,v,close,turnover,timestamp
0,BTCUSDT,0.0,0.346478,0.524043,0.15977,57989.4,4.288642e+08,1723816800000


In [8]:
from crypto_env import CryptoEnv

train_parameters = {
    "load_run": None,
    "train_frames": 0}

env = CryptoEnv(train_configs=train_parameters)

In [208]:
import numpy as np

symbol = "BTCUSDT"
timestamp = 1723816800000 + 60 * 60 * 1000
test_klines = env.load_klines(symbol, timestamp)

symbol_state = actor.state.symbol_states[symbol]

test_obs = np.hstack((test_klines, symbol_state['history']['action_history'], symbol_state['history']['cumulative_reward_history']))

actions, action_probs, q_action, v = actor.get_action_values(np.expand_dims(test_obs, axis=0))

print(actions, action_probs, q_action, v)

[1] [[0.346478]] [[0.5240429]] [[0.15976994]]


In [209]:
test_obs - observations

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0

In [192]:
test_obs[:,0]

array([1.14934313, 1.07618999, 0.87343168, 0.94120127, 0.87614763,
       0.86203676, 0.88535237, 0.80819792, 0.79280734, 0.93819809,
       0.93819618, 1.02813673, 1.03965807, 1.00857532, 1.05783069,
       1.0397681 , 0.98772615, 1.0046376 , 1.11394   , 1.15067935,
       1.20061731, 1.20294952, 1.36066103, 1.42758679, 1.35689545,
       1.40368509, 1.3716327 ])

In [193]:
observations[0,:,0]

array([1.07618999, 0.87343168, 0.94120127, 0.87614763, 0.86203676,
       0.88535237, 0.80819792, 0.79280734, 0.93819809, 0.93819618,
       1.02813673, 1.03965807, 1.00857532, 1.05783069, 1.0397681 ,
       0.98772615, 1.0046376 , 1.11394   , 1.15067935, 1.20061731,
       1.20294952, 1.36066103, 1.42758679, 1.35689545, 1.40368509,
       1.3716327 , 1.38436186])

In [171]:
a = np.array([1,2,3,4,5,6,7,8,9,10])
i = np.searchsorted(a, 8.5, side='left')
print(i, a[:i])

8 [1 2 3 4 5 6 7 8]


In [34]:
summary = actor.create_action_summary(actions, action_probs, q_action, v, last_klines)
len(summary)

231

In [35]:
chosen_symbols = actor.choose_symbols(summary)
len(chosen_symbols)

0

In [36]:
chosen_symbols

{}

In [13]:
chosen_actions = actor.update_trading_symbols(chosen_symbols)
len(chosen_actions)

3

In [ ]:
chosen_actions

In [ ]:
chosen_actions = await actor.run()
chosen_actions

## Testing Random Stuff

### instruments info

In [ ]:
import aiohttp

base_url = "https://api.bybit.com/v5/market"
category = "linear"

async def get_all_symbols():
    url = f"{base_url}/instruments-info"
    params = {"category": category}
    async with aiohttp.ClientSession() as session:
        async with session.get(url, params=params) as response:
            data = await response.json()
            if data['retCode'] != 0:
                print(f"Error fetching instrument info: {data['retMsg']}")
                return []
            return [
                {
                        'symbol': symbol['symbol'],
                        # 'tickSize': float(symbol['priceFilter']['tickSize'])
                        # 'all_data': symbol,
                        'qtyStep': symbol['lotSizeFilter']['qtyStep'],
                    }
                    for symbol in data.get('result', {}).get('list', [])
                    if symbol['quoteCoin'] == 'USDT'
                ]

symbols = await get_all_symbols()
len(symbols)

407

In [ ]:
symbols

### load numpy klines

In [ ]:
import os
import numpy as np
from datetime import datetime


def load_klines_up_to_time(symbol, target_time, n, interval="1 day"):
    # Convert target_time to Unix timestamp if it's a string
    if isinstance(target_time, str):
        target_timestamp = int(datetime.strptime(target_time, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
    else:
        target_timestamp = target_time

    # Construct file paths
    data_dir = f"../data/klines_processed/{interval}"  # Adjust this path as needed
    data_file = os.path.join(data_dir, f"{symbol}USDT.npy")
    timestamps_file = os.path.join(data_dir, f"{symbol}USDT_timestamps.npy")
    
    # Load data and timestamps
    data = np.load(data_file)
    timestamps = np.load(timestamps_file)
    
    # Find the index of the closest timestamp <= target_timestamp
    target_index = np.searchsorted(timestamps, target_timestamp, side='right') - 1
    
    # Calculate the start index
    start_index = max(0, target_index - n + 1)
    
    # Return the last n klines up to the target timestamp
    return data[start_index:target_index+1], timestamps[start_index:target_index+1]


# Example usage
symbol = "BTC"
target_time_unix = 1692144000000  # August 16, 2023 00:00:00 UTC
target_time_str = "2024-08-15 23:00:00"
n = 5

# Using Unix timestamp
klines1, timestamps1 = load_klines_up_to_time(symbol, target_time_unix, n)

# Using date-time string
klines2, timestamps2 = load_klines_up_to_time(symbol, target_time_str, n)

print(f"Loaded {len(klines1)} klines for {symbol} using Unix timestamp")
print(f"Loaded {len(klines2)} klines for {symbol} using date-time string")
print("\nLast timestamp (Unix):", timestamps1[-1])
print("Last timestamp (String):", timestamps2[-1])
print("\nLast kline data (Unix):", klines1[-1])
print("Last kline data (String):", klines2[-1])

Loaded 5 klines for BTC using Unix timestamp
Loaded 5 klines for BTC using date-time string

Last timestamp (Unix): 1692144000000
Last timestamp (String): 1723593600000

Last kline data (Unix): [0.639322   0.64027745 0.6321176  0.6323248  9.455806  ]
Last kline data (String): [0.9561997  0.9660658  0.9404286  0.94227344 9.944939  ]


In [ ]:
klines2

array([[ 0.9580324 ,  0.9626143 ,  0.9537848 ,  0.9585948 ,  9.487782  ],
       [ 0.9585948 ,  0.9655727 ,  0.9394227 ,  0.9426857 ,  9.777961  ],
       [ 0.9426857 ,  0.9571503 ,  0.9341055 ,  0.94719505, 10.067209  ],
       [ 0.94719505,  0.96343887,  0.9402865 ,  0.9561997 ,  9.913214  ],
       [ 0.9561997 ,  0.9660658 ,  0.9404286 ,  0.94227344,  9.944939  ]],
      dtype=float32)

In [ ]:
timestamps1

dtype('int64')

### kline function

In [ ]:
from data_handler import DataHandler
handler = DataHandler()

Main data folder created: ../data
Interval folders created successfully.


In [ ]:
symbol = "BCHUSDT"
interval = "1 day"
length = 3
handler.fetch_kline_data(symbol, interval, length=length)

In [ ]:
response = handler.session.get_kline(
                    category=handler.category,
                    symbol="BCHUSDT",
                    interval=handler.all_intervals["1 hour"],
                    start=1607904000000,
                    end=None,
                    limit=3
                )
len(response['result']['list'])

3

In [ ]:
response

### unix / datetime

In [105]:
from datetime import datetime, timezone

def date_to_unix_time(date_str):
    dt = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    dt = dt.replace(tzinfo=timezone.utc)
    return int(dt.timestamp() * 1000)

unix_time = 1726437600000
date = datetime.fromtimestamp(unix_time / 1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
print(f"{unix_time}  -->  {date}")

date_str = "2024-09-15 23:00:00"
unix_time_back = date_to_unix_time(date_str)
print(f"{date_str}  -->  {unix_time_back}")

1726437600000  -->  2024-09-15 22:00:00
2024-09-15 23:00:00  -->  1726441200000


### ccxt

In [ ]:
import configs_trader as configs

import ccxt

# Initialize the Bybit exchange
exchange = ccxt.bybit({
    'apiKey': 'configs.API_KEY',
    'secret': 'configs.API_SECRET',
    'enableRateLimit': True,
    'options': {
        'defaultType': 'linear',
        'adjustForTimeDifference': True,
        'timeDifference': 2000
    }
})

In [ ]:
markets = exchange.load_markets()

In [ ]:
# Fetch markets
markets = exchange.fetchMarkets()
len(markets)

2453

In [ ]:
markets[2]

In [ ]:
# Fetch markets
markets = exchange.load_markets()

# Get ticker for a specific symbol
symbol = 'BTC/USDT'
ticker = exchange.fetch_ticker(symbol)

print(f"Current price of {symbol}: {ticker['last']}")

# Fetch account balance
balance = exchange.fetch_balance()
print(f"USDT Balance: {balance['USDT']['free']}")

# Place a market order (be cautious with real accounts)
# order = exchange.create_market_buy_order(symbol, 0.001)  # Buy 0.001 BTC
# print(order)

Current price of BTC/USDT: 58299.2
USDT Balance: 53.91378734
